In [1]:
import pandas as pd
import numpy as np

# 可視化用のライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import neologdn
import MeCab

import re

from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc="progress: ")

In [2]:
df = pd.read_csv('all_text_wakatigaki.csv',encoding='utf-8')
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...


## TF値算出

In [3]:
# 単語名は同一でも品詞名が違う組み合わせがあるので単語名と品詞名を組み合わせた要素を格納した新規カラムを作成する
df['original_type'] = df['original'] + '_' + df['type']
df.head()

,surface,yomi,original,type,katsuyoukei,katsuyougata,内容,original_type
0,neologd,ネオログディー,NEologd,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般
1,_,_,_,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般
2,tagger,タガー,tagger,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般
3,.,.,.,記号-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般
4,parse,パース,Parse,名詞-固有名詞-一般,NaN,NaN,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般


In [4]:
# 必要カラムのみ抽出
df = df[['yomi', '内容', 'original_type']]
df.head()

,yomi,内容,original_type
0,ネオログディー,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般
1,_,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般
2,タガー,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般
3,.,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般
4,パース,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般


In [5]:
# [内容]×[original_type]の組み合わせの数をカウント
contents_word_cnt_df = df.groupby(['内容', 'original_type']).count().reset_index()
contents_word_cnt_df = contents_word_cnt_df.rename(columns={'yomi':'count'})
contents_word_cnt_df.head()

,内容,original_type,count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1


In [6]:
# 文書毎に含まれる単語数をカウント

# [内容]カラムをグループ化しカウント　
# reset_index関数を使い[内容]カラムをindex部分からデータ列部分に戻す
per_sentence_word_cnt = df.groupby('内容')['original_type'].count().reset_index()
per_sentence_word_cnt = per_sentence_word_cnt.rename(columns={'original_type':'word_count'})
per_sentence_word_cnt.head()

,内容,word_count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,26
1,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,30
2,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,33
3,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,41


In [7]:
# [count]と[word_count]を [内容]で紐づけ
contents_merge_df = pd.merge(contents_word_cnt_df, per_sentence_word_cnt, on='内容', how='inner')
contents_merge_df.head(27)

,内容,original_type,count,word_count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26
6,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般,1,26
7,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般,1,26
8,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,text_名詞-固有名詞-一般,1,26
9,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,、_記号-読点,1,26


In [8]:
# 欠損値があるか確認
contents_merge_df.isnull().any()

内容               False
original_type    False
count            False
word_count       False
dtype: bool

In [9]:
# tf値算出に必要なカラム（分子と分母の値があるカラム）のみ抽出
tmp_tf_df = contents_merge_df[['count', 'word_count']]
tmp_tf_df.head()

,count,word_count
0,1,26
1,1,26
2,1,26
3,1,26
4,1,26


In [10]:
# tf値算出
# axis=1をつけないと"occurred at index count"とエラーが発生する（デフォルトがaxis=0のため）
contents_merge_df['tf'] = tmp_tf_df.progress_apply(lambda x: x['count'] / x['word_count'], axis=1)

In [11]:
# tf値が紐づいたか確認
contents_merge_df.head()

,内容,original_type,count,word_count,tf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462


## IDF値算出

In [12]:
# idfの分子（全文書数）
len(contents_word_cnt_df['内容'].value_counts())

4

In [13]:
# idfの分母「各単語が出現する文書の数」の算出
# [original_type]×[内容]のクロス集計表を作成

idf_pivot_df = pd.pivot_table(contents_merge_df, index='original_type', columns='内容', aggfunc=len, values='count')
#tf_pivot_df = pd.pivot_table(contents_merge_df, index='original_type', columns='内容', values='count')#aggfuncを指定しないと[count]カラムの数字が入る
idf_pivot_df

内容,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない
original_type,,,,
')_記号-一般,NaN,NaN,NaN,1.0
('\_記号-一般,NaN,NaN,NaN,1.0
()_名詞-固有名詞-一般,NaN,1.0,NaN,NaN
(_記号-一般,1.0,NaN,NaN,NaN
)_記号-一般,1.0,NaN,NaN,NaN
._記号-一般,1.0,NaN,NaN,NaN
1_名詞-数,1.0,NaN,NaN,NaN
1つ_名詞-一般,NaN,1.0,NaN,NaN
NEologd_名詞-固有名詞-一般,1.0,NaN,NaN,NaN


In [14]:
idf_pivot_df1 = idf_pivot_df > 0
idf_pivot_df1# 1以上ならTrue、1未満ならFalse

内容,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない
original_type,,,,
')_記号-一般,False,False,False,True
('\_記号-一般,False,False,False,True
()_名詞-固有名詞-一般,False,True,False,False
(_記号-一般,True,False,False,False
)_記号-一般,True,False,False,False
._記号-一般,True,False,False,False
1_名詞-数,True,False,False,False
1つ_名詞-一般,False,True,False,False
NEologd_名詞-固有名詞-一般,True,False,False,False


In [15]:
# True＝１なのでTureの数を行毎に合計
# それを格納するカラム[word_include_text_count]を追加
# この値がidfの分母となる
idf_pivot_df1['word_include_text_count'] = idf_pivot_df1.sum(axis=1)
idf_pivot_df1

内容,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする,原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_textは文字列型ではなくリスト型のためsplit関数を使用できない,word_include_text_count
original_type,,,,,
')_記号-一般,False,False,False,True,1
('\_記号-一般,False,False,False,True,1
()_名詞-固有名詞-一般,False,True,False,False,1
(_記号-一般,True,False,False,False,1
)_記号-一般,True,False,False,False,1
._記号-一般,True,False,False,False,1
1_名詞-数,True,False,False,False,1
1つ_名詞-一般,False,True,False,False,1
NEologd_名詞-固有名詞-一般,True,False,False,False,1


In [16]:
# 必要なカラム(新規作成カラム、マージするときのキーのカラム）のみ抽出
idf_pivot_df2 = idf_pivot_df1.reset_index()[['original_type', 'word_include_text_count']]
idf_pivot_df2

内容,original_type,word_include_text_count
0,')_記号-一般,1
1,('\_記号-一般,1
2,()_名詞-固有名詞-一般,1
3,(_記号-一般,1
4,)_記号-一般,1
5,._記号-一般,1
6,1_名詞-数,1
7,1つ_名詞-一般,1
8,NEologd_名詞-固有名詞-一般,1
9,Parse_名詞-固有名詞-一般,2


In [17]:
# 新規に作成した「各単語が出現する文書の数」カラムを[original_type]をキーにして連結
idf_merge_df = pd.merge(contents_merge_df, idf_pivot_df2, on='original_type', how='inner')
idf_merge_df

,内容,original_type,count,word_count,tf,word_include_text_count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462,1
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462,1
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462,1
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462,1
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462,1
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26,0.038462,2
6,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,Parse_名詞-固有名詞-一般,1,30,0.033333,2
7,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般,1,26,0.038462,2
8,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,__記号-一般,1,41,0.024390,2
9,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般,1,26,0.038462,1


In [18]:
idf_merge_df.isnull().any()

内容                         False
original_type              False
count                      False
word_count                 False
tf                         False
word_include_text_count    False
dtype: bool

In [19]:
# dfの分子（全文書数）を格納する新規カラムを追加
idf_merge_df['all_text_count'] = len(contents_word_cnt_df['内容'].value_counts())
idf_merge_df

,内容,original_type,count,word_count,tf,word_include_text_count,all_text_count
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462,1,4
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462,1,4
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462,1,4
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462,1,4
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462,1,4
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26,0.038462,2,4
6,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,Parse_名詞-固有名詞-一般,1,30,0.033333,2,4
7,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般,1,26,0.038462,2,4
8,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,__記号-一般,1,41,0.024390,2,4
9,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般,1,26,0.038462,1,4


In [20]:
# idfを算出
# データフレーム型にapply関数を使う際には、axis=1を忘れないように注意
import math
idf_merge_df['idf'] = idf_merge_df.progress_apply(lambda x: math.log(x['all_text_count'] / x['word_include_text_count']) + 1 , axis=1)

In [21]:
idf_merge_df

,内容,original_type,count,word_count,tf,word_include_text_count,all_text_count,idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462,1,4,2.386294
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462,1,4,2.386294
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462,1,4,2.386294
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462,1,4,2.386294
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462,1,4,2.386294
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26,0.038462,2,4,1.693147
6,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,Parse_名詞-固有名詞-一般,1,30,0.033333,2,4,1.693147
7,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般,1,26,0.038462,2,4,1.693147
8,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,__記号-一般,1,41,0.024390,2,4,1.693147
9,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般,1,26,0.038462,1,4,2.386294


## TFIDF値を算出し新規カラムを作成

In [22]:
idf_merge_df['tf_idf'] = idf_merge_df['tf'] * idf_merge_df['idf']
idf_merge_df

,内容,original_type,count,word_count,tf,word_include_text_count,all_text_count,idf,tf_idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,1,26,0.038462,1,4,2.386294,0.091781
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,1,26,0.038462,1,4,2.386294,0.091781
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,1,26,0.038462,1,4,2.386294,0.091781
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,1,26,0.038462,1,4,2.386294,0.091781
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,1,26,0.038462,1,4,2.386294,0.091781
5,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,Parse_名詞-固有名詞-一般,1,26,0.038462,2,4,1.693147,0.065121
6,まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、...,Parse_名詞-固有名詞-一般,1,30,0.033333,2,4,1.693147,0.056438
7,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,__記号-一般,1,26,0.038462,2,4,1.693147,0.065121
8,次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid...,__記号-一般,1,41,0.024390,2,4,1.693147,0.041296
9,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,tagger_名詞-固有名詞-一般,1,26,0.038462,1,4,2.386294,0.091781


## テキスト内容×単語のデータフレーム を作成

In [23]:
idf_merge_df2 = idf_merge_df[['内容', 'original_type', 'tf_idf']]
idf_merge_df2.head()

,内容,original_type,tf_idf
0,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,(_記号-一般,0.091781
1,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,)_記号-一般,0.091781
2,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,._記号-一般,0.091781
3,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,1_名詞-数,0.091781
4,neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して...,NEologd_名詞-固有名詞-一般,0.091781


In [24]:
model_df = pd.pivot_table(idf_merge_df2, index='内容', columns='original_type')
# nanは0に変換する
model_df = model_df.fillna(0)
model_df

tf_idf            \
original_type                                       ')_記号-一般 ('\_記号-一般   
内容                                                                       
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...  0.000000  0.000000   
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...  0.000000  0.000000   
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される      0.000000  0.000000   
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...  0.058202  0.058202   

                                                                            \
original_type                                      ()_名詞-固有名詞-一般   (_記号-一般   
内容                                                                           
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...      0.000000  0.091781   
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...      0.079543  0.000000   
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される          0.000000  0.000000   
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...      0.000000  0.000000   

                                                                        \
original_type                                        )_記号-一般   ._記号-一般   
内容                                                                       
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...  0.091781  0.091781   
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...  0.000000  0.000000   
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される      0.000000  0.000000   
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...  0.000000  0.000000   

                                                                        \
original_type                                         1_名詞-数  1つ_名詞-一般   
内容                                                                       
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...  0.091781  0.000000   
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...  0.000000  0.079543   
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される      0.000000  0.000000   
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...  0.000000  0.000000   

                                                                       \
original_type                                      NEologd_名詞-固有名詞-一般   
内容                                                                      
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...           0.091781   
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...           0.000000   
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される               0.000000   
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...           0.000000   

                                                                       ...     \
original_type                                      Parse_名詞-固有名詞-一般    ...      
内容                                                                     ...      
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...         0.065121    ...      
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...         0.056438    ...      
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される             0.000000    ...      
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...         0.000000    ...      

                                                                             \
original_type                                      文字列_名詞-固有名詞-一般    次に_接続詞   
内容                                                                            
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...       0.000000  0.000000   
まずはparse()で分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リ...       0.056438  0.000000   
原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される           0.000000  0.000000   
次に邪魔な文字「\t」を省きたい。よって、split('\t')を使用したいがwakatid_...       0.041296  0.058202   

                                                                         \
original_type                                       省く_動詞-自立 群_名詞-接尾-一般   
内容                                                                        
neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表...  0.000000   0.000000   
まずはparse()で分かち書き